In [21]:
import load_env
import src.graphql.models.recipe_models_autogen as rma
from sqlalchemy import select
from src.graphql.db.session import get_session
from src.graphql.scalars.recipe_scalar import NEI_Item, NEI_Fluid, NEI_Recipe_Dimensions, NEI_GT_Recipe
from src.graphql.resolvers.recipe_resolver import _getNEIRecipe

In [33]:
# Test joins
recipe_id = 'r~--eEYPHsN5GqWygrEHan4A=='

# stmt = select(rma.Item, rma.RecipeItemOutputs) \
#         .join(rma.RecipeItemOutputs, rma.RecipeItemOutputs.item_outputs_value_item_id == rma.Item.id) \
#         .filter(rma.RecipeItemOutputs.recipe_id == recipe_id)
stmt = select(rma.GregTechRecipe, rma.RecipeType) \
        .join(rma.Recipe, rma.GregTechRecipe.recipe_id == rma.Recipe.id) \
        .join(rma.RecipeType, rma.Recipe.recipe_type_id == rma.RecipeType.id) \
        .filter(rma.GregTechRecipe.recipe_id == recipe_id)

print(stmt)

SELECT greg_tech_recipe.id, greg_tech_recipe.additional_info, greg_tech_recipe.amperage, greg_tech_recipe.duration, greg_tech_recipe.requires_cleanroom, greg_tech_recipe.requires_low_gravity, greg_tech_recipe.voltage, greg_tech_recipe.voltage_tier, greg_tech_recipe.recipe_id, recipe_type.id AS id_1, recipe_type.category, recipe_type.fluid_input_dimension_height, recipe_type.fluid_input_dimension_width, recipe_type.fluid_output_dimension_height, recipe_type.fluid_output_dimension_width, recipe_type.icon_info, recipe_type.item_input_dimension_height, recipe_type.item_input_dimension_width, recipe_type.item_output_dimension_height, recipe_type.item_output_dimension_width, recipe_type.shapeless, recipe_type.type, recipe_type.icon_id 
FROM greg_tech_recipe JOIN recipe ON greg_tech_recipe.recipe_id = recipe.id JOIN recipe_type ON recipe.recipe_type_id = recipe_type.id 
WHERE greg_tech_recipe.recipe_id = :recipe_id_1


In [37]:
def _prepORMDict(item, rename={}, exclude=[], include={}):
    d = dict(item.__dict__)
    d.pop('_sa_instance_state')

    for k, v in rename.items():
        d[v] = d.pop(k)
    for k in exclude:
        d.pop(k)
    d.update(include)
    
    return d


async with get_session() as session:
    rows = await session.execute(stmt)
    gt_recipes = []
    for r in rows:
        gtRecipeORM, recipeTypeORM = r
        findict = _prepORMDict(
            recipeTypeORM,
            rename={
                'type': 'localized_machine_name',
            },
            exclude = [
                'category', # They're all "gregtech"
                'id'
            ],
        )
        findict.update(_prepORMDict(
            gtRecipeORM,
            rename = {
                'duration': 'duration_ticks',
            },
            exclude = ['id'],
        ))
        for single_type in ['item', 'fluid']:
            for direction in ['input', 'output']:
                findict[f'{single_type}_{direction}_dims'] = NEI_Recipe_Dimensions(
                    height = findict.pop(f'{single_type}_{direction}_dimension_height'),
                    width = findict.pop(f'{single_type}_{direction}_dimension_width'),
                )
        findict['base_recipe'] = await _getNEIRecipe(session, recipe_id)

        gt_recipes.append(
            NEI_GT_Recipe(**findict)
        )

    # print(result)
    # for r in result:
    #     print(r.__dict__)

    print(gt_recipes)

TypeError: NEI_GT_Recipe.__init__() missing 1 required keyword-only argument: 'base_recipe'